In [17]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
import re

In [3]:
llm = ChatOllama(model="llama3.2")

def run_prompt(prompt, **kwargs):
    """Helper function to run a prompt through the language model.

    Args:
        prompt (str): The prompt template string.
        **kwargs: Keyword arguments to fill the prompt template.

    Returns:
        str: The model's response.
    """
    prompt_template = PromptTemplate(template=prompt, input_variables=list(kwargs.keys()))
    chain = prompt_template | llm
    return chain.invoke(kwargs).content

# Task Decomposition

## Breaking Down Complex Tasks

In [9]:
complex_task = """
Analyze the financial health of a company based on the following data:
- Revenue: $10 million
- Net Income: $2 million
- Total Assets: $15 million
- Total Liabilities: $7 million
- Cash Flow from Operations: $3 million
"""

decomposition_prompt = """
Break down the task of analyzing a company's financial health into 3 subtasks. For each subtask, provide a brief description of what it should accomplish.

Task: {task}

Subtasks:
1.
"""

subtasks = run_prompt(decomposition_prompt, task=complex_task)
print(subtasks)

Here are the 3 subtasks to analyze a company's financial health, along with brief descriptions of what each subtask should accomplish:

**Subtask 1: Assess Liquidity and Solvency**

* Evaluate the company's ability to meet its short-term obligations (less than 1 year) using various liquidity ratios such as:
	+ Current Ratio (Current Assets / Total Liabilities)
	+ Quick Ratio (Current Assets - Inventory / Total Liabilities)
* Analyze the company's debt-to-equity ratio and net debt (Total Liabilities - Cash Reserves) to understand its financial leverage and ability to service debts.

**Subtask 2: Evaluate Profitability**

* Calculate key profitability ratios such as:
	+ Gross Margin Ratio (Gross Profit / Revenue)
	+ Operating Margin Ratio (Operating Income / Revenue)
	+ Net Profit Margin Ratio (Net Income / Revenue)
* Analyze the company's return on equity (ROE) to understand its ability to generate profits from shareholder investments.

**Subtask 3: Assess Cash Flow and Efficiency**

* 

## Chaining Subtasks in Prompts

In [10]:
def analyze_profitability(revenue, net_income):
    """Analyze the company's profitability.

    Args:
        revenue (float): Company's revenue.
        net_income (float): Company's net income.

    Returns:
        str: Analysis of the company's profitability.
    """
    prompt = """
    Analyze the company's profitability based on the following data:
    - Revenue: ${revenue} million
    - Net Income: ${net_income} million

    Calculate the profit margin and provide a brief analysis of the company's profitability.
    """
    return run_prompt(prompt, revenue=revenue, net_income=net_income)

def analyze_liquidity(total_assets, total_liabilities):
    """Analyze the company's liquidity.

    Args:
        total_assets (float): Company's total assets.
        total_liabilities (float): Company's total liabilities.

    Returns:
        str: Analysis of the company's liquidity.
    """
    prompt = """
    Analyze the company's liquidity based on the following data:
    - Total Assets: ${total_assets} million
    - Total Liabilities: ${total_liabilities} million

    Calculate the current ratio and provide a brief analysis of the company's liquidity.
    """
    return run_prompt(prompt, total_assets=total_assets, total_liabilities=total_liabilities)

def analyze_cash_flow(cash_flow):
    """Analyze the company's cash flow.

    Args:
        cash_flow (float): Company's cash flow from operations.

    Returns:
        str: Analysis of the company's cash flow.
    """
    prompt = """
    Analyze the company's cash flow based on the following data:
    - Cash Flow from Operations: ${cash_flow} million

    Provide a brief analysis of the company's cash flow health.
    """
    return run_prompt(prompt, cash_flow=cash_flow)

# Run the chained subtasks
profitability_analysis = analyze_profitability(10, 2)
liquidity_analysis = analyze_liquidity(15, 7)
cash_flow_analysis = analyze_cash_flow(3)

print("Profitability Analysis:\n", profitability_analysis)
print("\nLiquidity Analysis:\n", liquidity_analysis)
print("\nCash Flow Analysis:\n", cash_flow_analysis)

Profitability Analysis:
 To calculate the profit margin, we can use the following formula:

Profit Margin = (Net Income / Revenue) x 100

Plugging in the values provided, we get:

Profit Margin = ($2,000,000 / $10,000,000) x 100
= 0.20 x 100
= 20%

Based on this calculation, the company's profit margin is 20%. This means that for every dollar of revenue generated, the company retains 20 cents as net income.

A profit margin of 20% indicates that the company is profitable, but it may also suggest some inefficiencies or areas for improvement. Here are a few possible interpretations:

1. **Limited pricing power**: A profit margin of 20% suggests that the company's prices may not be competitive in the market, as it would need to charge significantly higher prices to achieve similar profitability.
2. **Efficiency and cost management**: Despite the relatively low profit margin, the company appears to be profitable, indicating that it has managed costs effectively. However, there may still be

## Integrating Results

In [11]:
def integrate_results(profitability, liquidity, cash_flow):
    """Integrate the results from subtasks to provide an overall analysis.

    Args:
        profitability (str): Profitability analysis.
        liquidity (str): Liquidity analysis.
        cash_flow (str): Cash flow analysis.

    Returns:
        str: Overall analysis of the company's financial health.
    """
    prompt = """
    Based on the following analyses, provide an overall assessment of the company's financial health:

    Profitability Analysis:
    {profitability}

    Liquidity Analysis:
    {liquidity}

    Cash Flow Analysis:
    {cash_flow}

    Summarize the key points and give an overall evaluation of the company's financial position.
    """
    return run_prompt(prompt, profitability=profitability, liquidity=liquidity, cash_flow=cash_flow)

overall_analysis = integrate_results(profitability_analysis, liquidity_analysis, cash_flow_analysis)
print("Overall Financial Health Analysis:\n", overall_analysis)

Overall Financial Health Analysis:
 Based on the provided analyses, here is a summary of the key points:

**Key Points:**

1. **Profitability:** The company has a profit margin of 20%, indicating that it is profitable but may have some inefficiencies or areas for improvement.
2. **Liquidity:** The current ratio is 1.14, which suggests that the company has sufficient liquidity to meet its short-term obligations.
3. **Cash Flow:** The company's cash flow from operations is $3 million, which may be sufficient to cover operational expenses but may not be enough for long-term growth or investments.

**Overall Evaluation:**

The company appears to have a relatively stable financial position, with some strengths and weaknesses. Here are the overall observations:

* **Positive:** The company has managed costs effectively, indicating that it is profitable and has a solid foundation for future growth.
* **Negative:** The profit margin of 20% may suggest that prices are not competitive in the mar

# Prompt Chaining and Sequencing

## Basic Prompt Chaining


In [12]:
# Define prompt templates
story_prompt = PromptTemplate(
    input_variables=["genre"],
    template="Write a short {genre} story in 3-4 sentences."
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize the following story in one sentence:\n{story}"
)

# Chain the prompts
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary

# Test the chain
genre = "science fiction"
story, summary = story_chain(genre)
print(f"Story: {story}\n\nSummary: {summary}")

Story: As the last star in the universe died, Dr. Maria Rodriguez stood alone on the desolate planet of Elyria, surrounded by an eternal darkness that seemed to have a life of its own. The once-blue skies now hung as a dull grey membrane, suffocating all life beneath it. In her mind, she heard the whispers of her ancestors, guiding her towards a secret she had spent her entire career searching for: a way to reignite the very fabric of existence itself. With a heavy heart, Maria reached into the depths of Elyria and set the universe ablaze once more.

Summary: Dr. Maria Rodriguez, standing alone on the desolate planet of Elyria, finds the strength to reignite the universe after it dies, guided by her ancestors' whispers and driven by a lifelong quest for knowledge.


## Sequential Prompting

In [14]:
# Define prompt templates for each step
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"
)

def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme of the text is the responsible development and use of artificial intelligence (AI), highlighting both its potential benefits and risks, and emphasizing the need for careful consideration and planning to mitigate potential negative consequences.

Tone: The overall tone of the text is cautious and thoughtful. The author presents a balanced view of artificial intelligence, acknowledging both its potential benefits and drawbacks. They express concern for the ethical implications of AI, but also emphasize the importance of responsible development and use.

The tone is not overly alarmist or negative, as phrases such as "spark[ing] excitement" suggest that the author believes in the potential of AI to improve our lives. However, the text does convey a sense of caution and responsibility, particularly in the phrase "approach AI development with caution and foresight."

Overall, the tone is measured and reflective, encouraging readers to consider the complexities of AI de

## Dynamic Prompt Generation



In [15]:
# Define prompt templates
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

# Test the dynamic Q&A system
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: Quantum computing has numerous potential applications, including:

1. Cryptography and cybersecurity
2. Optimization problems (e.g., logistics, finance)
3. Materials science and chemistry research
4. Machine learning and artificial intelligence
5. Simulating complex systems (e.g., weather forecasting, materials properties)
6. Quantum simulation of chemical reactions
7. Breaking encryption algorithms
8. Medical imaging and disease diagnosis
9. Optimization of complex networks (e.g., traffic flow, energy grids)

These applications are still being researched and developed, but quantum computing has the potential to revolutionize various fields with its unique capabilities.

Q2: "What are some potential challenges and limitations that researchers and developers will need to address as they explore and implement these applications of quantum computing?"
A2: Potential challenges and limitations in quantum computing application

## Error Handling and Validation


In [18]:
# Define prompt templates
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")
            
            # Validate the relevance
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
    
    return "Failed to generate a valid number after multiple attempts."

# Test the robust number generation
topic = "World War II"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'World War II': 1945


# Instruction Engineering

In [19]:
def get_completion(prompt):
    """Helper function to get model completion."""
    return llm.invoke(prompt).content

## Clear Instructions

In [20]:
vague_instruction = "Tell me about climate change concisely."
clear_instruction = "Provide a concise summary of the primary causes and effects of climate change, focusing on scientific consensus from the past five years conciesly."

print("Vague Instruction Output:")
print(get_completion(vague_instruction))

print("\nClear Instruction Output:")
print(get_completion(clear_instruction))

Vague Instruction Output:
Here's a concise overview of climate change:

**What is Climate Change?**

Climate change refers to the long-term warming of the planet due to an increase in average global temperature, primarily caused by human activities.

**Causes:**

1. **Greenhouse gases**: Burning fossil fuels (coal, oil, gas), deforestation, and land-use changes release CO2, methane, and other greenhouse gases.
2. **Population growth**: Increasing energy demand and consumption.
3. **Industrialization**: Agriculture, transportation, and urbanization.

**Effects:**

1. **Rising sea levels**: Melting glaciers and ice sheets.
2. **Extreme weather**: Heatwaves, droughts, floods, and storms.
3. **Changes in precipitation patterns**: More frequent and intense rain or snow events.
4. **Loss of biodiversity**: Rising temperatures threaten ecosystems and extinction.

**Consequences:**

1. **Food insecurity**: Changes in temperature and precipitation affect crop yields.
2. **Water scarcity**: Decr

## Effective Instruction Structures


In [22]:
bullet_structure = """
Explain the process of photosynthesis conciesly:
- Define photosynthesis
- List the main components involved
- Describe the steps in order
- Mention its importance for life on Earth
"""

narrative_structure = """
Imagine you're a botanist explaining photosynthesis to a curious student. 
Start with a simple definition, then walk through the process step-by-step, 
highlighting the key components involved. Conclude by emphasizing why 
photosynthesis is crucial for life on Earth. Write it concisely.
"""

print("Bullet Structure Output:")
print(get_completion(bullet_structure))

print("\nNarrative Structure Output:")
print(get_completion(narrative_structure))

Bullet Structure Output:
**Definition:** Photosynthesis is a process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose.

**Main Components Involved:**

1. Chlorophyll (a green pigment that absorbs light energy)
2. Water
3. Carbon dioxide
4. Light energy from the sun

**Steps in Order:**

1. **Light absorption**: Chlorophyll absorbs light energy from the sun and transfers it to a molecule called ATP (adenosine triphosphate).
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through small openings on their leaves called stomata.
4. **Light-dependent reactions**: The absorbed light energy is used to power a series of chemical reactions that produce ATP and another molecule called NADPH (nicotinamide adenine dinucleotide phosphate).
5. **Calvin cycle**: The ATP and NADPH produced in the pre

## Balancing Specificity and Generality

In [23]:
specific_instruction = """
Describe the plot of the 1985 film 'Back to the Future', focusing on:
1. The main character's name and his friendship with Dr. Brown
2. The time machine and how it works
3. The specific year the main character travels to and why it's significant
4. The main conflict involving his parents' past
5. How the protagonist resolves the issues and returns to his time
Limit your response to 150 words. 
"""

general_instruction = """
Describe the plot of a popular time travel movie from the 1980s. Include:
1. The main characters and their relationships
2. The method of time travel
3. The time period visited and its significance
4. The main conflict or challenge faced
5. How the story is resolved
Keep your response around 150 words.
"""

print("Specific Instruction Output:")
print(get_completion(specific_instruction))

print("\nGeneral Instruction Output:")
print(get_completion(general_instruction))

Specific Instruction Output:
In 'Back to the Future', Marty McFly, a high school student, befriends eccentric scientist Dr. Emmett Brown. They create a time machine using a DeLorean car. The vehicle operates at 88 miles per hour, reaching the time-travel threshold. Marty travels back to November 5, 1955, where he meets his parents as teenagers.

The main conflict arises when Marty's presence in the past threatens to alter the course of his own birth. His father, George, is a bully who will eventually kill Biff Tannen, Marty's future tormentor. To fix this, Marty must convince his parents to fall in love and get married. He also helps his dad gain confidence.

Marty resolves the issue by returning to 1985, where his actions have created a better future for himself. The time machine is repaired, and Marty resumes life in 1985 with improved relationships and a more secure future.

General Instruction Output:
The movie "Back to the Future" (1985) follows Marty McFly, a high school student 

## Iterative Refinement


In [24]:
initial_instruction = "Explain how to make a peanut butter and jelly sandwich."

print("Initial Instruction Output:")
initial_output = get_completion(initial_instruction)
print(initial_output)

refined_instruction = """
Explain how to make a peanut butter and jelly sandwich, with the following improvements:
1. Specify the type of bread, peanut butter, and jelly to use
2. Include a step about washing hands before starting
3. Mention how to deal with potential allergies
4. Add a tip for storing the sandwich if not eaten immediately
Present the instructions in a numbered list format.
"""

print("\nRefined Instruction Output:")
refined_output = get_completion(refined_instruction)
print(refined_output)

Initial Instruction Output:
Making a peanut butter and jelly sandwich is a simple yet delicious process that requires just a few ingredients and some basic steps. Here's a step-by-step guide:

Ingredients:

* 2 slices of bread
* Peanut butter (crunchy or smooth, whichever you prefer)
* Jelly or jam (your favorite flavor)
* A knife for spreading the peanut butter and jelly
* Optional: a plate for assembling the sandwich

Instructions:

1. **Lay out your bread**: Place two slices of bread on a clean, flat surface.
2. **Spread the peanut butter**: Take your knife and scoop up a small amount of peanut butter from the jar. Spread it evenly onto one slice of bread, covering most of the surface. You can apply gentle pressure to ensure an even spread.
3. **Spread the jelly**: Repeat step 2 with the jelly, scooping up a similar amount to the peanut butter and spreading it evenly onto the second slice of bread.
4. **Assemble the sandwich**: Carefully place the slice of bread with the jelly on to